In [2]:
!pip install fuzzywuzzy

In [1]:
!pip install python-Levenshtein

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.6 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 5.8 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz, process

# Path to files
base_path = "C:/Users/w196283/Downloads/cleaned/"
files = {
    'hot100': base_path + "hot100_transformed.csv",
    'radio': base_path + "radio_transformed.csv",
    'streaming': base_path + "streaming_transformed.csv",
    'sales': base_path + "sales_transformed.csv",
    'apple': base_path + "apple_music_transformed.csv",
    'spotify': base_path + "spotify_transformed.csv"
}

# Load all data files
hot100_df = pd.read_csv(files['hot100'])
radio_df = pd.read_csv(files['radio'])
streaming_df = pd.read_csv(files['streaming'])
sales_df = pd.read_csv(files['sales'])
apple_df = pd.read_csv(files['apple'])
spotify_df = pd.read_csv(files['spotify'])

print("Data loaded successfully")
print(f"Hot100: {len(hot100_df)} rows")
print(f"Radio: {len(radio_df)} rows")
print(f"Streaming: {len(streaming_df)} rows")
print(f"Sales: {len(sales_df)} rows")
print(f"Apple: {len(apple_df)} rows")
print(f"Spotify: {len(spotify_df)} rows")

# Improved function to clean text for matching
def clean_for_matching(text):
    """Normalize text for fuzzy matching"""
    if not isinstance(text, str):
        return ""
    
    # Convert to lowercase
    text = text.lower().strip()
    
    # Replace various forms of "featuring"
    text = re.sub(r'(\sfeat\.|\sfeaturing|\sft\.|\swith|\sand|\s&|\svs\.|\svs\s).*', '', text)
    
    # Remove text in parentheses and brackets
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'\[[^\]]*\]', '', text)
    
    # Replace punctuation and special characters with spaces
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Remove specific words that might cause confusion
    text = re.sub(r'\b(the|a|an)\b', '', text)
    
    # Replace multiple spaces with single space
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Create a fresh unified dataset starting with Hot100
unified = hot100_df.copy()

# Based on the data you showed, here are the actual column mappings:
print("\nRenaming Hot100 columns...")
unified = unified.rename(columns={
    'rank': 'hot100_rank',
    'bb_last_week': 'hot100_last_week',
    'bb_peak': 'hot100_peak',
    'bb_woc': 'hot100_woc'
})

# Ensure label column exists
if 'label' not in unified.columns:
    unified['label'] = ''

# Create normalized columns for matching
print("\nCreating normalized matching columns...")
unified['track_clean'] = unified['track'].apply(clean_for_matching)
unified['artist_clean'] = unified['artist'].apply(clean_for_matching)

radio_df['track_clean'] = radio_df['track'].apply(clean_for_matching)
radio_df['artist_clean'] = radio_df['artist'].apply(clean_for_matching)

streaming_df['track_clean'] = streaming_df['track'].apply(clean_for_matching)
streaming_df['artist_clean'] = streaming_df['artist'].apply(clean_for_matching)

sales_df['track_clean'] = sales_df['track'].apply(clean_for_matching)
sales_df['artist_clean'] = sales_df['artist'].apply(clean_for_matching)

apple_df['track_clean'] = apple_df['track'].apply(clean_for_matching)
apple_df['artist_clean'] = apple_df['artist'].apply(clean_for_matching)

spotify_df['track_clean'] = spotify_df['track'].apply(clean_for_matching)
spotify_df['artist_clean'] = spotify_df['artist'].apply(clean_for_matching)

# Initialize all platform columns with zeros
print("\nInitializing platform columns...")
# Radio columns
unified['radio_rank'] = 0
unified['radio_last_week'] = 0
unified['radio_peak'] = 0
unified['radio_woc'] = 0
unified['radio_change'] = 0.0

# Streaming columns
unified['streaming_rank'] = 0
unified['streaming_last_week'] = 0
unified['streaming_peak'] = 0
unified['streaming_woc'] = 0
unified['streaming_change'] = 0.0

# Sales columns
unified['sales_rank'] = 0
unified['sales_last_week'] = 0
unified['sales_peak'] = 0
unified['sales_woc'] = 0
unified['sales_change'] = 0.0

# Apple columns
unified['apple_rank'] = 0
unified['apple_last_week'] = 0
unified['apple_peak'] = 0
unified['apple_woc'] = 0
unified['apple_change'] = 0.0

# Spotify columns
unified['spotify_rank'] = 0
unified['spotify_last_week'] = 0
unified['spotify_peak'] = 0
unified['spotify_woc'] = 0
unified['spotify_change'] = 0.0
unified['spotify_streams'] = 0

# For Hot100, add change column
unified['hot100_change'] = 0.0

# Function to find matches with fallback to fuzzy matching
def find_match(hot_row, platform_week_df, threshold=85):
    # Try direct match first
    match = platform_week_df[(platform_week_df['track_clean'] == hot_row['track_clean']) & 
                           (platform_week_df['artist_clean'] == hot_row['artist_clean'])]
    
    if not match.empty:
        return match.iloc[0]
    
    # Try matching just by track name if artist is the same
    match = platform_week_df[(platform_week_df['artist_clean'] == hot_row['artist_clean'])]
    if not match.empty:
        # Find best track match
        for _, row in match.iterrows():
            ratio = fuzz.token_sort_ratio(row['track_clean'], hot_row['track_clean'])
            if ratio >= threshold:
                return row
    
    # Try matching by track name with fuzzy artist matching
    match = platform_week_df[platform_week_df['track_clean'] == hot_row['track_clean']]
    if not match.empty:
        # Find best artist match
        for _, row in match.iterrows():
            ratio = fuzz.token_sort_ratio(row['artist_clean'], hot_row['artist_clean'])
            if ratio >= threshold:
                return row
    
    # If still no match, try fuzzy matching on both track and artist
    for _, row in platform_week_df.iterrows():
        track_ratio = fuzz.token_sort_ratio(row['track_clean'], hot_row['track_clean'])
        artist_ratio = fuzz.token_sort_ratio(row['artist_clean'], hot_row['artist_clean'])
        
        # Combined score gives more weight to track name
        combined_ratio = (track_ratio * 0.7) + (artist_ratio * 0.3)
        
        if combined_ratio >= threshold:
            return row
    
    return None

# Match and copy data by week
print("\nMatching songs and copying data...")

# 1. Radio data
print("Processing RADIO data:")
for week in unified['week'].unique():
    unified_week = unified[unified['week'] == week]
    radio_week = radio_df[radio_df['week'] == week]
    
    if radio_week.empty:
        print(f"  Week {week}: No radio data")
        continue
    
    matches = 0
    for idx, hot_row in unified_week.iterrows():
        match_row = find_match(hot_row, radio_week)
        
        if match_row is not None:
            # Copy rank
            unified.loc[idx, 'radio_rank'] = match_row['rank']
            
            # Copy other metrics
            unified.loc[idx, 'radio_last_week'] = match_row['radio_last_week']
            unified.loc[idx, 'radio_peak'] = match_row['radio_peak']
            unified.loc[idx, 'radio_woc'] = match_row['radio_woc']
            
            # Copy label if available
            if 'label' in match_row and pd.notnull(match_row['label']) and match_row['label'] != '':
                if pd.isnull(unified.loc[idx, 'label']) or unified.loc[idx, 'label'] == '':
                    unified.loc[idx, 'label'] = match_row['label']
            
            matches += 1
    
    print(f"  Week {week}: Matched {matches} of {len(unified_week)} songs")

# Print verification
radio_matched = (unified['radio_rank'] > 0).sum()
print(f"RADIO data verification: {radio_matched} songs with non-zero rank")

# 2. Streaming data
print("\nProcessing STREAMING data:")
for week in unified['week'].unique():
    unified_week = unified[unified['week'] == week]
    streaming_week = streaming_df[streaming_df['week'] == week]
    
    if streaming_week.empty:
        print(f"  Week {week}: No streaming data")
        continue
    
    matches = 0
    for idx, hot_row in unified_week.iterrows():
        match_row = find_match(hot_row, streaming_week)
        
        if match_row is not None:
            # Copy rank
            unified.loc[idx, 'streaming_rank'] = match_row['rank']
            
            # Copy other metrics
            unified.loc[idx, 'streaming_last_week'] = match_row['str_last_week']
            unified.loc[idx, 'streaming_peak'] = match_row['str_peak']
            unified.loc[idx, 'streaming_woc'] = match_row['str_woc']
            
            # Copy label if available
            if 'label' in match_row and pd.notnull(match_row['label']) and match_row['label'] != '':
                if pd.isnull(unified.loc[idx, 'label']) or unified.loc[idx, 'label'] == '':
                    unified.loc[idx, 'label'] = match_row['label']
            
            matches += 1
    
    print(f"  Week {week}: Matched {matches} of {len(unified_week)} songs")

# Print verification
streaming_matched = (unified['streaming_rank'] > 0).sum()
print(f"STREAMING data verification: {streaming_matched} songs with non-zero rank")

# 3. Sales data
print("\nProcessing SALES data:")
for week in unified['week'].unique():
    unified_week = unified[unified['week'] == week]
    sales_week = sales_df[sales_df['week'] == week]
    
    if sales_week.empty:
        print(f"  Week {week}: No sales data")
        continue
    
    matches = 0
    for idx, hot_row in unified_week.iterrows():
        match_row = find_match(hot_row, sales_week)
        
        if match_row is not None:
            # Copy rank
            unified.loc[idx, 'sales_rank'] = match_row['rank']
            
            # Copy other metrics
            unified.loc[idx, 'sales_last_week'] = match_row['sales_last_week']
            unified.loc[idx, 'sales_peak'] = match_row['sales_peak']
            unified.loc[idx, 'sales_woc'] = match_row['sales_woc']
            
            # Copy label if available
            if 'label' in match_row and pd.notnull(match_row['label']) and match_row['label'] != '':
                if pd.isnull(unified.loc[idx, 'label']) or unified.loc[idx, 'label'] == '':
                    unified.loc[idx, 'label'] = match_row['label']
            
            matches += 1
    
    print(f"  Week {week}: Matched {matches} of {len(unified_week)} songs")

# Print verification
sales_matched = (unified['sales_rank'] > 0).sum()
print(f"SALES data verification: {sales_matched} songs with non-zero rank")

# 4. Apple data
print("\nProcessing APPLE data:")
for week in unified['week'].unique():
    unified_week = unified[unified['week'] == week]
    apple_week = apple_df[apple_df['week'] == week]
    
    if apple_week.empty:
        print(f"  Week {week}: No Apple data")
        continue
    
    matches = 0
    for idx, hot_row in unified_week.iterrows():
        match_row = find_match(hot_row, apple_week)
        
        if match_row is not None:
            # Copy rank
            unified.loc[idx, 'apple_rank'] = match_row['rank']
            
            # Copy other metrics
            unified.loc[idx, 'apple_last_week'] = match_row['am_last_week']
            unified.loc[idx, 'apple_peak'] = match_row['am_peak']
            unified.loc[idx, 'apple_woc'] = match_row['am_woc']
            
            # Copy label if available
            if 'label' in match_row and pd.notnull(match_row['label']) and match_row['label'] != '':
                if pd.isnull(unified.loc[idx, 'label']) or unified.loc[idx, 'label'] == '':
                    unified.loc[idx, 'label'] = match_row['label']
            
            matches += 1
    
    print(f"  Week {week}: Matched {matches} of {len(unified_week)} songs")

# Print verification
apple_matched = (unified['apple_rank'] > 0).sum()
print(f"APPLE data verification: {apple_matched} songs with non-zero rank")

# 5. Spotify data
print("\nProcessing SPOTIFY data:")
for week in unified['week'].unique():
    unified_week = unified[unified['week'] == week]
    spotify_week = spotify_df[spotify_df['week'] == week]
    
    if spotify_week.empty:
        print(f"  Week {week}: No Spotify data")
        continue
    
    matches = 0
    for idx, hot_row in unified_week.iterrows():
        match_row = find_match(hot_row, spotify_week)
        
        if match_row is not None:
            # Copy rank
            unified.loc[idx, 'spotify_rank'] = match_row['rank']
            
            # Copy other metrics
            unified.loc[idx, 'spotify_last_week'] = match_row['sp_last_week']
            unified.loc[idx, 'spotify_peak'] = match_row['sp_peak']
            unified.loc[idx, 'spotify_woc'] = match_row['sp_woc']
            unified.loc[idx, 'spotify_streams'] = match_row['streams']
            
            # Copy label if available
            if 'label' in match_row and pd.notnull(match_row['label']) and match_row['label'] != '':
                if pd.isnull(unified.loc[idx, 'label']) or unified.loc[idx, 'label'] == '':
                    unified.loc[idx, 'label'] = match_row['label']
            
            matches += 1
    
    print(f"  Week {week}: Matched {matches} of {len(unified_week)} songs")

# Print verification
spotify_matched = (unified['spotify_rank'] > 0).sum()
print(f"SPOTIFY data verification: {spotify_matched} songs with non-zero rank")

# Final verification
print("\nFINAL VERIFICATION:")
print(f"Total rows: {len(unified)}")
print(f"Hot100 songs: {(unified['hot100_rank'] > 0).sum()}")
print(f"Radio songs: {(unified['radio_rank'] > 0).sum()}")
print(f"Streaming songs: {(unified['streaming_rank'] > 0).sum()}")
print(f"Sales songs: {(unified['sales_rank'] > 0).sum()}")
print(f"Apple songs: {(unified['apple_rank'] > 0).sum()}")
print(f"Spotify songs: {(unified['spotify_rank'] > 0).sum()}")

# Strategic label filling - after matching but before feature generation
print("\nPerforming strategic label filling...")

# 1. First pass: Fill labels based on the same song appearing in different weeks
print("Step 1: Filling labels based on temporal data...")
# Group by track and artist
song_groups = unified.groupby(['track_clean', 'artist_clean'])

temporal_filled = 0
for (track, artist), group in song_groups:
    # Check if this song has a label in at least one week
    labels = group['label'].dropna().unique()
    labels = [l for l in labels if l and l != 'Unknown']
    
    if len(labels) > 0:
        # Use the first non-empty label
        valid_label = labels[0]
        
        # Apply this label to all weeks where this song appears but has no label
        for idx in group.index:
            if pd.isna(unified.loc[idx, 'label']) or unified.loc[idx, 'label'] == '' or unified.loc[idx, 'label'] == 'Unknown':
                unified.loc[idx, 'label'] = valid_label
                temporal_filled += 1

print(f"Filled {temporal_filled} labels using temporal matching")

# 2. Second pass: Use artist's most common label
print("Step 2: Filling labels based on artist's common label...")
artist_labels = {}
for idx, row in unified.iterrows():
    if pd.notna(row['label']) and row['label'] != '' and row['label'] != 'Unknown':
        artist_clean = row['artist_clean']
        if artist_clean not in artist_labels:
            artist_labels[artist_clean] = []
        
        artist_labels[artist_clean].append(row['label'])

# Find most common label for each artist
artist_most_common_label = {}
for artist, labels in artist_labels.items():
    if not labels:
        continue
    
    label_counts = {}
    for label in labels:
        if label not in label_counts:
            label_counts[label] = 0
        label_counts[label] += 1
    
    most_common = max(label_counts.items(), key=lambda x: x[1])
    artist_most_common_label[artist] = most_common[0]

# Apply artist's most common label to songs missing labels
artist_filled = 0
for idx, row in unified.iterrows():
    if pd.isna(row['label']) or row['label'] == '' or row['label'] == 'Unknown':
        if row['artist_clean'] in artist_most_common_label:
            unified.loc[idx, 'label'] = artist_most_common_label[row['artist_clean']]
            artist_filled += 1

print(f"Filled {artist_filled} labels using artist's common label")

# 3. Third pass: Fill any remaining empty labels with 'Unknown'
empty_before = (unified['label'].isna() | (unified['label'] == '')).sum()
unified['label'] = unified['label'].fillna('Unknown')
unified.loc[unified['label'] == '', 'label'] = 'Unknown'
empty_after = (unified['label'].isna() | (unified['label'] == '')).sum()

print(f"Filled {empty_before - empty_after} remaining empty labels with 'Unknown'")
print(f"Songs with known labels: {(unified['label'] != 'Unknown').sum()} of {len(unified)}")

# Add derived features and clean the data
print("\nAdding derived features...")
# Calculated columns for each platform
platforms = {
    'hot100': 100,
    'radio': 50,
    'streaming': 50,
    'sales': 25,
    'apple': 100,
    'spotify': 200
}

# 1. Is collaboration
unified['is_collab'] = unified['artist'].str.contains(
    r'featuring|feat\.|ft\.|&|,|with|and', case=False, regex=True).astype(int)

# 2. Calculate changes for each platform
print("Calculating platform changes...")
for platform, size in platforms.items():
    rank_col = f'{platform}_rank'
    last_week_col = f'{platform}_last_week'
    change_col = f'{platform}_change'
    
    # Make sure all these columns exist 
    if rank_col in unified.columns and last_week_col in unified.columns and change_col in unified.columns:
        print(f"  Calculating {change_col}")
        
        # Both weeks on chart
        mask = (unified[rank_col] > 0) & (unified[last_week_col] > 0)
        unified.loc[mask, change_col] = ((unified.loc[mask, last_week_col] - 
                                        unified.loc[mask, rank_col]) / size * 100).round(2)
        
        # New entries
        mask = (unified[rank_col] > 0) & (unified[last_week_col] == 0)
        unified.loc[mask, change_col] = ((size + 1 - unified.loc[mask, rank_col]) / size * 100).round(2)
        
        # Dropped off
        mask = (unified[rank_col] == 0) & (unified[last_week_col] > 0)
        unified.loc[mask, change_col] = ((unified.loc[mask, last_week_col] - 
                                        (size + 1)) / size * 100).round(2)
    else:
        print(f"  Missing columns for {platform} change calculation")

# 3. Calculate platforms available
print("Calculating available platforms...")
rank_cols = [f'{platform}_rank' for platform in platforms.keys() if f'{platform}_rank' in unified.columns]
unified['available_platforms'] = unified[rank_cols].apply(lambda row: (row > 0).sum(), axis=1)

# 4. Determine best platform
print("Determining best platform...")
unified['best_platform'] = ''
for idx, row in unified.iterrows():
    if row['available_platforms'] >= 1:
        best_pos = float('inf')
        best_platform = ''
        
        for platform, size in platforms.items():
            rank_col = f'{platform}_rank'
            if rank_col in unified.columns:
                rank = row[rank_col]
                if rank > 0 and rank < best_pos:
                    best_pos = rank
                    best_platform = platform
        
        unified.at[idx, 'best_platform'] = best_platform

# 5. Calculate overall momentum
print("Calculating platform momentum...")
change_cols = [f'{platform}_change' for platform in platforms.keys() if f'{platform}_change' in unified.columns]
if change_cols:
    unified['platform_momentum'] = unified[change_cols].replace(0, np.nan).mean(axis=1, skipna=True).round(2)
    unified['platform_momentum'] = unified['platform_momentum'].fillna(0)
else:
    unified['platform_momentum'] = 0.0
    print("  Warning: No change columns found for momentum calculation")

# Clean data
print("\nCleaning data...")
# Ensure numeric columns have correct data types
for platform in platforms.keys():
    # Rank columns
    rank_col = f'{platform}_rank'
    if rank_col in unified.columns:
        unified[rank_col] = pd.to_numeric(unified[rank_col], errors='coerce').fillna(0).astype(int)
    
    # Last week columns
    last_week_col = f'{platform}_last_week'
    if last_week_col in unified.columns:
        unified[last_week_col] = pd.to_numeric(unified[last_week_col], errors='coerce').fillna(0).astype(int)
        unified[last_week_col] = unified[last_week_col].replace(-1, 0)
    
    # Peak columns
    peak_col = f'{platform}_peak'
    if peak_col in unified.columns:
        unified[peak_col] = pd.to_numeric(unified[peak_col], errors='coerce').fillna(0).astype(int)
    
    # WOC columns
    woc_col = f'{platform}_woc'
    if woc_col in unified.columns:
        unified[woc_col] = pd.to_numeric(unified[woc_col], errors='coerce').fillna(0).astype(int)
    
    # Change columns
    change_col = f'{platform}_change'
    if change_col in unified.columns:
        unified[change_col] = pd.to_numeric(unified[change_col], errors='coerce').fillna(0)

# Spotify streams
if 'spotify_streams' in unified.columns:
    unified['spotify_streams'] = pd.to_numeric(unified['spotify_streams'], errors='coerce').fillna(0).astype(int)

# Add advanced features
print("\nAdding advanced features...")
final_df = unified.copy()

# 1. Platform comparisons
final_df['apple_vs_spotify'] = 0.0
final_df['radio_vs_streaming'] = 0.0
final_df['streaming_vs_sales'] = 0.0

# Apple vs Spotify
if 'apple_rank' in final_df.columns and 'spotify_rank' in final_df.columns:
    mask = (final_df['apple_rank'] > 0) & (final_df['spotify_rank'] > 0)
    final_df.loc[mask, 'apple_vs_spotify'] = (
        (final_df.loc[mask, 'spotify_rank'] / 200) - 
        (final_df.loc[mask, 'apple_rank'] / 100)
    ).round(2)

# Radio vs Streaming
if 'radio_rank' in final_df.columns and 'streaming_rank' in final_df.columns:
    mask = (final_df['radio_rank'] > 0) & (final_df['streaming_rank'] > 0)
    final_df.loc[mask, 'radio_vs_streaming'] = (
        (final_df.loc[mask, 'streaming_rank'] / 50) - 
        (final_df.loc[mask, 'radio_rank'] / 50)
    ).round(2)

# Streaming vs Sales
if 'streaming_rank' in final_df.columns and 'sales_rank' in final_df.columns:
    mask = (final_df['streaming_rank'] > 0) & (final_df['sales_rank'] > 0)
    final_df.loc[mask, 'streaming_vs_sales'] = (
        (final_df.loc[mask, 'sales_rank'] / 25) - 
        (final_df.loc[mask, 'streaming_rank'] / 50)
    ).round(2)

# 2. Top artists and labels by week
final_df['is_top_artist'] = 0
final_df['is_top_label'] = 0

for week in final_df['week'].unique():
    weekly_data = final_df[final_df['week'] == week]
    
    # TOP ARTISTS
    artist_scores = {}
    for artist in weekly_data['artist'].unique():
        artist_data = weekly_data[weekly_data['artist'] == artist]
        song_count = artist_data['track'].nunique()
        avg_position = 101 - artist_data['hot100_rank'].mean()
        best_rank = artist_data['hot100_rank'].min()
        
        score = song_count * 50 + avg_position * 0.5 + (101 - best_rank)
        artist_scores[artist] = score
    
    top_artists = sorted(artist_scores.items(), key=lambda x: x[1], reverse=True)[:10]
    for artist, _ in top_artists:
        final_df.loc[(final_df['week'] == week) & (final_df['artist'] == artist), 'is_top_artist'] = 1
    
    # TOP LABELS
    if 'label' in final_df.columns:
        label_scores = {}
        for label in weekly_data['label'].unique():
            if pd.isna(label) or label == '' or label == 'Unknown':
                continue
            
            label_data = weekly_data[weekly_data['label'] == label]
            song_count = label_data['track'].nunique()
            avg_position = 101 - label_data['hot100_rank'].mean()
            top10_count = label_data[label_data['hot100_rank'] <= 10].shape[0]
            
            score = song_count * 30 + avg_position + top10_count * 20
            label_scores[label] = score
        
        top_labels = sorted(label_scores.items(), key=lambda x: x[1], reverse=True)[:10]
        for label, _ in top_labels:
            final_df.loc[(final_df['week'] == week) & (final_df['label'] == label), 'is_top_label'] = 1

# Clean up temporary columns
final_df = final_df.drop(columns=['track_clean', 'artist_clean'])

# Rearrange columns in the specified order
print("\nRearranging columns...")

# Define the column order
column_order = [
    # First group: quarter, month, week
    'quarter', 'month', 'week',
    
    # Second group: track, artist, label
    'track', 'artist', 'label',
    
    # Third group: all hot100 columns
    'hot100_rank', 'hot100_last_week', 'hot100_peak', 'hot100_woc', 'hot100_change',
]

# Add all remaining columns that aren't explicitly listed above
remaining_columns = [col for col in final_df.columns if col not in column_order]
column_order.extend(remaining_columns)

# Reorder the columns
final_df = final_df[column_order]

# Save the final dataset
print("\nSaving final dataset...")
final_df.to_csv(base_path + "charts_final_enhanced.csv", index=False)
print("Final dataset saved successfully!")

# Final verification
print("\nFINAL VERIFICATION OF ENHANCED DATASET:")
print(f"Total rows: {len(final_df)}")
print(f"Hot100 songs: {(final_df['hot100_rank'] > 0).sum()}")
print(f"Radio songs: {(final_df['radio_rank'] > 0).sum()}")
print(f"Streaming songs: {(final_df['streaming_rank'] > 0).sum()}")
print(f"Sales songs: {(final_df['sales_rank'] > 0).sum()}")
print(f"Apple songs: {(final_df['apple_rank'] > 0).sum()}")
print(f"Spotify songs: {(final_df['spotify_rank'] > 0).sum()}")
print(f"Songs with known labels: {(final_df['label'] != 'Unknown').sum()} of {len(final_df)}")

Data loaded successfully
Hot100: 5200 rows
Radio: 2600 rows
Streaming: 2600 rows
Sales: 1300 rows
Apple: 5200 rows
Spotify: 10400 rows

Renaming Hot100 columns...

Creating normalized matching columns...

Initializing platform columns...

Matching songs and copying data...
Processing RADIO data:
  Week 1: Matched 38 of 100 songs
  Week 2: Matched 43 of 100 songs
  Week 3: Matched 46 of 100 songs
  Week 4: Matched 43 of 100 songs
  Week 5: Matched 47 of 100 songs
  Week 6: Matched 47 of 100 songs
  Week 7: Matched 46 of 100 songs
  Week 8: Matched 43 of 100 songs
  Week 9: Matched 46 of 100 songs
  Week 10: Matched 45 of 100 songs
  Week 11: Matched 46 of 100 songs
  Week 12: Matched 46 of 100 songs
  Week 13: Matched 44 of 100 songs
  Week 14: Matched 40 of 100 songs
  Week 15: Matched 37 of 100 songs
  Week 16: Matched 40 of 100 songs
  Week 17: Matched 40 of 100 songs
  Week 18: Matched 36 of 100 songs
  Week 19: Matched 36 of 100 songs
  Week 20: Matched 37 of 100 songs
  Week 21: M

In [2]:
# Script for generating advanced features
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv("charts_final_enhanced.csv")
print("Generating advanced features...")

# Platform list
platforms = ['hot100', 'radio', 'streaming', 'sales', 'apple', 'spotify']

# 1. Seasonal variables
print("Adding seasonal variables...")
df['is_holiday_season'] = ((df['week'] >= 47) | (df['week'] == 1)).astype(int)
df['is_summer_season'] = ((df['month'] >= 6) & (df['month'] <= 8)).astype(int)
df['is_award_season'] = ((df['month'] == 2) | (df['month'] == 9) | (df['month'] == 10)).astype(int)

# First convert all change variables to decimal format (0.30 instead of 30%)
# We do this BEFORE calculating acceleration to ensure consistency
print("Converting change variables to decimal format...")
change_cols = [col for col in df.columns if 'change' in col]
for col in change_cols:
    if df[col].abs().max() > 1:  # If values are large, they're likely percentages
        df[col] = (df[col] / 100).round(2)

# 2. Growth rate for each platform
print("Calculating growth rate...")
for platform in platforms:
    rank_col = f'{platform}_rank'
    last_week_col = f'{platform}_last_week'
    growth_col = f'{platform}_growth_rate'
    
    df[growth_col] = 0.0
    
    # Only calculate for tracks present in both weeks
    mask = (df[rank_col] > 0) & (df[last_week_col] > 0)
    
    # Formula: (previous_position - current_position) / previous_position
    df.loc[mask, growth_col] = (
        (df.loc[mask, last_week_col] - df.loc[mask, rank_col]) / 
        df.loc[mask, last_week_col]
    ).round(2)

# 3. Acceleration for each platform - now calculated AFTER change conversion
print("Calculating acceleration...")
for platform in platforms:
    change_col = f'{platform}_change'
    accel_col = f'{platform}_acceleration'
    
    # Group by track and calculate difference between consecutive changes
    df[accel_col] = df.groupby(['track', 'artist'])[change_col].diff().fillna(0).round(2)

# 4. Divergence index
print("Calculating divergence index...")
normalized_ranks = []
for platform in platforms:
    rank_col = f'{platform}_rank'
    
    # Set chart size for normalization
    if platform == 'hot100': max_rank = 100
    elif platform in ['radio', 'streaming']: max_rank = 50
    elif platform == 'sales': max_rank = 25
    elif platform == 'apple': max_rank = 100
    elif platform == 'spotify': max_rank = 200
    else: continue
    
    # Normalize rank for valid positions
    norm_col = f'{platform}_rank_norm'
    df[norm_col] = 0.0
    mask = df[rank_col] > 0
    df.loc[mask, norm_col] = df.loc[mask, rank_col] / max_rank
    normalized_ranks.append(norm_col)

# Calculate divergence as standard deviation of normalized ranks
if normalized_ranks:
    df['platform_divergence'] = df[normalized_ranks].std(axis=1, skipna=True).round(2)
    df['platform_divergence'] = df['platform_divergence'].fillna(0)
    
    # Clean up temporary columns
    df = df.drop(columns=normalized_ranks)

# 5. Calculate platform momentum metrics with proper naming
print("Calculating platform momentum metrics...")

# Change momentum (renaming from platform_momentum)
change_cols = [f'{platform}_change' for platform in platforms if f'{platform}_change' in df.columns]
if change_cols:
    # If platform_momentum already exists, rename it
    if 'platform_momentum' in df.columns:
        df['platform_change_momentum'] = df['platform_momentum']
        df = df.drop(columns=['platform_momentum'])
    else:
        # Calculate platform_change_momentum
        df['platform_change_momentum'] = df[change_cols].replace(0, np.nan).mean(axis=1, skipna=True).round(2)
        df['platform_change_momentum'] = df['platform_change_momentum'].fillna(0)

# Acceleration momentum
accel_cols = [f'{platform}_acceleration' for platform in platforms if f'{platform}_acceleration' in df.columns]
if accel_cols:
    df['platform_acceleration_momentum'] = df[accel_cols].replace(0, np.nan).mean(axis=1, skipna=True).round(2)
    df['platform_acceleration_momentum'] = df['platform_acceleration_momentum'].fillna(0)

# 6. Recalculate comparison columns in consistent order
print("Recalculating platform comparisons in consistent order...")

# Remove old comparison columns if they exist
old_comparison_cols = ['apple_vs_spotify', 'radio_vs_streaming', 'streaming_vs_sales']
for col in old_comparison_cols:
    if col in df.columns:
        df = df.drop(columns=[col])

# Calculate new comparison columns in consistent order
# Spotify vs Apple
if 'spotify_rank' in df.columns and 'apple_rank' in df.columns:
    mask = (df['spotify_rank'] > 0) & (df['apple_rank'] > 0)
    df['spotify_vs_apple'] = 0.0
    df.loc[mask, 'spotify_vs_apple'] = (
        (df.loc[mask, 'apple_rank'] / 100) - 
        (df.loc[mask, 'spotify_rank'] / 200)
    ).round(2)

# Streaming vs Radio
if 'streaming_rank' in df.columns and 'radio_rank' in df.columns:
    mask = (df['streaming_rank'] > 0) & (df['radio_rank'] > 0)
    df['streaming_vs_radio'] = 0.0
    df.loc[mask, 'streaming_vs_radio'] = (
        (df.loc[mask, 'radio_rank'] / 50) - 
        (df.loc[mask, 'streaming_rank'] / 50)
    ).round(2)

# Streaming vs Sales
if 'streaming_rank' in df.columns and 'sales_rank' in df.columns:
    mask = (df['streaming_rank'] > 0) & (df['sales_rank'] > 0)
    df['streaming_vs_sales'] = 0.0
    df.loc[mask, 'streaming_vs_sales'] = (
        (df.loc[mask, 'sales_rank'] / 25) - 
        (df.loc[mask, 'streaming_rank'] / 50)
    ).round(2)

# Add Radio vs Sales
if 'radio_rank' in df.columns and 'sales_rank' in df.columns:
    mask = (df['radio_rank'] > 0) & (df['sales_rank'] > 0)
    df['radio_vs_sales'] = 0.0
    df.loc[mask, 'radio_vs_sales'] = (
        (df.loc[mask, 'sales_rank'] / 25) - 
        (df.loc[mask, 'radio_rank'] / 50)
    ).round(2)

# 7. Lead-lag indicators with improved handling of edge cases
print("Calculating lead-lag indicators...")
# Modified to remove hot100-related lead-lag indicators
platform_pairs = [
    ('spotify', 'apple'),
    ('streaming', 'radio'),
    ('streaming', 'sales'),
    ('radio', 'sales')  # Added this pair
]

for p1, p2 in platform_pairs:
    p1_change = f'{p1}_change'
    p2_change = f'{p2}_change'
    indicator_col = f'{p1}_leads_{p2}'
    
    # Initialize column
    df[indicator_col] = 0.0
    
    # Calculate for each track separately
    for (track, artist), group in df.groupby(['track', 'artist']):
        if len(group) < 3:  # Need at least 3 weeks for meaningful correlation
            continue
        
        # Get valid rows with non-zero changes
        valid_group = group[(~group[p1_change].isna()) & (~group[p2_change].isna()) & 
                           (group[p1_change] != 0) & (group[p2_change] != 0)]
        
        if len(valid_group) < 3:
            continue
            
        # Calculate correlation between p1 change in current week and p2 change in next week
        p1_current = valid_group[p1_change].values[:-1]
        p2_next = valid_group[p2_change].values[1:]
        
        if len(p1_current) >= 2 and np.any(p1_current != 0) and np.any(p2_next != 0):
            # Check for variation in the data to avoid division by zero
            if np.std(p1_current) > 0 and np.std(p2_next) > 0:
                try:
                    correlation = np.corrcoef(p1_current, p2_next)[0, 1]
                    if not np.isnan(correlation):
                        df.loc[group.index, indicator_col] = round(correlation, 2)
                except:
                    # Skip if correlation calculation fails
                    continue

# Organize columns in the requested order
print("Reorganizing columns...")

# Define column order groups
base_cols = ['quarter', 'month', 'week', 'track', 'artist', 'label']

hot100_cols = ['hot100_rank', 'hot100_last_week', 'hot100_peak', 'hot100_woc', 
               'hot100_change', 'hot100_growth_rate', 'hot100_acceleration']

radio_cols = ['radio_rank', 'radio_last_week', 'radio_peak', 'radio_woc', 
              'radio_change', 'radio_growth_rate', 'radio_acceleration']

streaming_cols = ['streaming_rank', 'streaming_last_week', 'streaming_peak', 'streaming_woc', 
                 'streaming_change', 'streaming_growth_rate', 'streaming_acceleration']

sales_cols = ['sales_rank', 'sales_last_week', 'sales_peak', 'sales_woc', 
             'sales_change', 'sales_growth_rate', 'sales_acceleration']

apple_cols = ['apple_rank', 'apple_last_week', 'apple_peak', 'apple_woc', 
             'apple_change', 'apple_growth_rate', 'apple_acceleration']

spotify_cols = ['spotify_rank', 'spotify_last_week', 'spotify_peak', 'spotify_woc', 
               'spotify_change', 'spotify_growth_rate', 'spotify_acceleration', 'spotify_streams']

additional_cols = ['is_collab', 'is_top_artist', 'is_top_label', 
                  'is_holiday_season', 'is_summer_season', 'is_award_season',
                  'available_platforms', 'best_platform', 'platform_change_momentum', 'platform_acceleration_momentum',
                  'platform_divergence']

# Updated comparison cols to match lead-lag order
comparison_cols = ['spotify_vs_apple', 'streaming_vs_radio', 'streaming_vs_sales', 'radio_vs_sales']

leadlag_cols = ['spotify_leads_apple', 'streaming_leads_radio', 'streaming_leads_sales', 'radio_leads_sales']

# Combine all column groups in the desired order
all_cols = (base_cols + hot100_cols + radio_cols + streaming_cols + 
            sales_cols + apple_cols + spotify_cols + 
            additional_cols + comparison_cols + leadlag_cols)

# Filter to only include columns that exist in the dataframe
final_cols = [col for col in all_cols if col in df.columns]

# Reorder the dataframe
df = df[final_cols]

# Save to final CSV
output_path = "final.csv"
df.to_csv(output_path, index=False)
print(f"Final dataset saved to {output_path}")

print("Advanced feature generation completed successfully.")

Generating advanced features...
Adding seasonal variables...
Converting change variables to decimal format...
Calculating growth rate...
Calculating acceleration...
Calculating divergence index...
Calculating platform momentum metrics...
Recalculating platform comparisons in consistent order...
Calculating lead-lag indicators...
Reorganizing columns...
Final dataset saved to final.csv
Advanced feature generation completed successfully.
